In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from numpy.linalg import norm as norm 
from numpy.linalg import pinv
from collections import Counter

 
from sklearn.preprocessing import PolynomialFeatures
from numpy.polynomial import chebyshev as cheb

from tqdm import tqdm 
import time

In [ ]:

X = np.linspace(-1., 1., 1001)
k = 100
nk = 2*(k+1)
M = cheb.chebvander(X, deg=nk)

obs_set = np.arange(k+1)
rand_state = np.random.RandomState(10)

# try ctrue zero everywhere in unmodeled space
# try having only a few in modeled be non-zero
# try random in unmodeled. vary size of values
# sparse in the whole thing but have a few in unmodeled have extra non-zeros
ctrue = np.zeros(nk+1)
ctrue[2*k ] = 1.5
# ctrue[2*k+1] = 1.0
ctrue[1] = 0.5
ctrue += rand_state.randn(ctrue.size)

f = np.array(M @ ctrue)

chat_true = np.zeros_like(ctrue)
sol = np.linalg.lstsq(M[:,obs_set], f, rcond=None)
chat_true[obs_set] = sol[0]
nstart = 5
max_samples = 200

In [ ]:
basis_idx = np.arange(ctrue.size)

def run_simulation(samples, probs=None, nstart=5):
    start_time = time.time()
    if probs is None:
        probs = np.ones_like(samples, dtype=float)

    assert probs.size == samples.size

    res = {'chat':[], 'errs':[], 'A_norm':[], 'MTMinv_norm':[], 'runtime':[]}
    T = samples[:nstart]

    for i in range(samples.size - nstart+1):
        MTM = M[np.ix_(T,obs_set)]
        
        MTU = M[np.ix_(T,np.delete(np.arange(M.shape[1]), obs_set))]
        probsi = np.sqrt(probs[:nstart+i]*(nstart+i))

        y = M[T,:] @ ctrue 
        sol = np.linalg.lstsq(MTM/probsi.reshape(-1, 1), y/probsi, rcond=None)
        chat = np.zeros_like(ctrue, dtype=float)
        chat[obs_set] = sol[0]
        res['chat'].append(chat) 
        res['errs'].append(np.linalg.norm(M@(ctrue - chat)))
        
        try: 
            Minv = pinv(MTM)
            res['MTMinv_norm'].append(norm(Minv,ord=2))
            res['A_norm'].append(norm(Minv @ MTU, ord=2))

        except np.linalg.LinAlgError:
            res['MTMinv_norm'].append(np.nan)
            res['A_norm'].append(np.nan)
        res['runtime'].append(time.time() - start_time)
        T = samples[:nstart+i+1]        
    return res 


def run_simulation_nodes(ntotal, nstart=5, snap_to_grid=False):
    start_time = time.time()
    Tx = cheb.chebpts2(nstart)
    if snap_to_grid:
        N = int(1 / (1-np.cos(np.pi / nstart)) + 1)
        grid = np.linspace(-1,1,N)
        Tx = np.array([grid[np.argmin(np.abs(grid - x))] for x in Tx])

    res = {'chat':[], 'errs':[], 'A_norm':[], 'MTMinv_norm':[], 'runtime':[]}
    
    
    for i in range(ntotal - nstart):
        M_ = cheb.chebvander(Tx, deg=nk)
        MTM = M_[:,obs_set]

        MTU = M_[:,np.delete(np.arange(M_.shape[1]), obs_set)]
        y = M_ @ ctrue
        sol = np.linalg.lstsq(MTM, y, rcond=None)
        chat = np.zeros_like(ctrue, dtype=float)
        chat[obs_set] = sol[0]
        res['chat'].append(chat) 
        res['errs'].append(np.linalg.norm(M@(chat_true - chat)))
        
        try: 
            Minv = pinv(MTM)
            res['MTMinv_norm'].append(norm(Minv,ord=2))
            res['A_norm'].append(norm(Minv @ MTU, ord=2))

        except np.linalg.LinAlgError:
            res['MTMinv_norm'].append(np.nan)
            res['A_norm'].append(np.nan)

        Tx = cheb.chebpts2(nstart+i+1)
        if snap_to_grid:
            N = int(1 / (1-np.cos(np.pi / nstart)) + 1)
            grid = np.linspace(-1,1,N)
            Tx = np.array([grid[np.argmin(np.abs(grid - x))] for x in Tx])
        res['runtime'].append(time.time() - start_time)    
    return res 

import numpy as np
from numpy.linalg import norm, pinv
import numpy.polynomial.chebyshev as cheb
    

In [ ]:
def run_simulation_nodes_incremental(ntotal, nstart=5):
    start_time = time.time()
    pts = cheb.chebpts2(ntotal)

    N = int(1 / (1-np.cos(np.pi / ntotal)) + 1) 
    print(N)
    grid = np.linspace(-1,1,N)
    chebyshev_grid = np.array([grid[np.argmin(np.abs(grid - x))] for x in pts])

    # print(f"Counts of all numbers: {dict(Counter(chebyshev_grid))}")
    random_state = np.random.RandomState(19)
    sample = random_state.choice(chebyshev_grid, ntotal, replace=True)
    
    res = {'chat':[], 'errs':[], 'A_norm':[], 'MTMinv_norm':[], 'runtime':[]}
    Tx = sample[:nstart]
    for i in range(ntotal - nstart + 1):
        M_ = cheb.chebvander(Tx, deg=nk)
        MTM = M_[:,obs_set]

        MTU = M_[:,np.delete(np.arange(M_.shape[1]), obs_set)]
        y = M_ @ ctrue
        sol = np.linalg.lstsq(MTM, y, rcond=None)
        chat = np.zeros_like(ctrue, dtype=float)
        chat[obs_set] = sol[0]
        res['chat'].append(chat) 
        res['errs'].append(np.linalg.norm(M@(chat_true - chat)))
        
        try: 
            Minv = pinv(MTM)
            res['MTMinv_norm'].append(norm(Minv,ord=2))
            res['A_norm'].append(norm(Minv @ MTU, ord=2))

        except np.linalg.LinAlgError:
            res['MTMinv_norm'].append(np.nan)
            res['A_norm'].append(np.nan)

        Tx = sample[:nstart+i+1]
        res['runtime'].append(time.time() - start_time)
    return res, sample

In [ ]:
# Chebyshev nodes
output = run_simulation_nodes_incremental(max_samples, nstart=nstart)
RESULTS_CHEB, points_cheb = [output[0]], output[1]


In [ ]:
from optimal_design_sub_mod import run_simulation_greedy_sub_mod
output = run_simulation_greedy_sub_mod(M, obs_set, ctrue, nstart, max_samples, "A")
RESULTS_OD_SUB_MOD_A, points_od_sub_mod_A = [output[0]], output[1]

In [ ]:
output = run_simulation_greedy_sub_mod(M, obs_set, ctrue, nstart, max_samples, "V")
RESULTS_OD_SUB_MOD_V, points_od_sub_mod_V = [output[0]], output[1]


In [ ]:
nsims = 10

In [ ]:
RESULTS_RAND = []
for j in tqdm(range(nsims), total=nsims):
    rand_state = np.random.RandomState(45+j)
    samples = np.arange(X.size)
    rand_state.shuffle(samples)
    samples = samples[:max_samples]
    RESULTS_RAND.append(run_simulation(samples, nstart=nstart))
points_rand = M[:,1][samples]

In [ ]:
Qs, _ = np.linalg.qr(M[:,obs_set])
poly_ls_probs_s = np.linalg.norm(Qs, axis=1)**2.
print(poly_ls_probs_s.max(), poly_ls_probs_s.min(), poly_ls_probs_s.sum())
poly_ls_probs_s /= poly_ls_probs_s.sum()

RESULTS_LS = []
RESULTS_LS_UNWEIGHTED = []
for j in tqdm(range(nsims), total=nsims):
    rand_state = np.random.RandomState(45+j)
    poly_ls_samples_s = rand_state.choice(X.size, max_samples, p=poly_ls_probs_s)
    poly_ls_probs_s_on_samples = poly_ls_probs_s[poly_ls_samples_s]
    RESULTS_LS.append(run_simulation(poly_ls_samples_s, poly_ls_probs_s_on_samples, nstart=nstart))
    RESULTS_LS_UNWEIGHTED.append(run_simulation(poly_ls_samples_s, nstart=nstart))

points_ls = M[:,1][poly_ls_samples_s]

In [ ]:
np.sum(poly_ls_probs_s)

In [ ]:
plt.plot(X, poly_ls_probs_s)

In [ ]:
save = True


keys = ['errs', 'MTMinv_norm', 'runtime']
for key in keys:
    if key == 'errs':
        ylabel = r"Error, $\|\hat{f}_{\mathcal{T}} - f\|_2$" 
    elif key == 'A_norm':
        ylabel = r"$\|A\|$"
    elif key == 'MTMinv_norm':
        ylabel = r"$\|M_{\mathcal{TM}}^\dagger\|$"
    elif key == 'runtime':
        ylabel = 'Time (s)'

    savename = f"{key.lower()}_rough_snap.png"
    fig, ax = plt.subplots()
    for results, name in zip([RESULTS_RAND, RESULTS_LS, RESULTS_CHEB, RESULTS_OD_SUB_MOD_V, RESULTS_OD_SUB_MOD_A], 
                             ['Random', 'Leverage Score', 'Chebyshev Nodes', 'Greedy Sub Modular OD V', 'Greedy Sub Modular OD A']):
        metric = np.array([res[key] for res in results])
        if key != 'runtime':
            metric /= X.size 
            
        mean = metric.mean(axis=0)

        line = ax.loglog(np.arange(nstart, max_samples+1), mean, label=name)[0]



    ax.legend(title='Sampling Method', fontsize=11)
    ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)
    if save:
        plt.savefig(savename, dpi=250, format='png')

In [ ]:
RESULTS_LS[2]['runtime']

In [ ]:
colors = ['red', 'green', 'purple', 'orange','pink', 'black']

In [ ]:
methods = [RESULTS_RAND, RESULTS_LS, RESULTS_CHEB, RESULTS_OD_SUB_MOD_V, RESULTS_OD_SUB_MOD_A]
names = ['Random', 'Leverage Score', 'Chebyshev Nodes', 'OD Sub Modular V', 'OD Sub Modular A']


fig, axes = plt.subplots(len(methods), 1, figsize=(8, 15), sharex=True)

for ax, method, name, color in zip(axes, methods, names, colors):
    ax.plot(X, M @ method[0]['chat'][95], label=name, color=color, alpha=0.6)
    ax.plot(X, M @ ctrue, label='True function', color='blue', alpha=0.8)
    ax.set_title(f"{name} vs. True Function at 100 Sampled Points")

    ax.legend()

plt.tight_layout()
plt.savefig('chat_at_100.png')

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, M@RESULTS_CHEB[0]['chat'][99], label='Greedy Sub Mod', color='red', alpha=0.4)
ax.plot(X, M@ctrue,marker='.',label='True function', color='blue', alpha=0.4)
plt.title("Estimated Function vs. True Function at 100 Sampled Points")
plt.legend()
plt.show()

In [ ]:
# collect cumulative runtime data at each value of k
runtimes = {'Continuous Relaxation'}

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(8, 2.5*4), sharex=True)
for i, dist, name, color in zip(np.arange(len(distributions)), distributions, names, colors):
    axes[i].hist(dist, bins=75, range=(-1,1), color=color, alpha=0.7, label=name)
    axes[i].set_ylabel("Count")
    axes[i].grid(alpha=0.3)

axes[-1].set_xlabel("Value")
plt.suptitle("Histograms of Each Distribution", fontsize=14)

plt.tight_layout()
fig.legend(
    loc="lower center",  
    bbox_to_anchor=(0.5, 0.12),         
    title="Distributions"
)
plt.savefig('final_point_distributions.png')